# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import sys
sys.path.append('C:/Users/User/Desktop/AG News')
from data_preprocessing import *

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

# 전처리

In [3]:
train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

#각 클래스별로 5000개씩 총 2만개의 데이터를 샘플랭(너무 크면 TextCuboid의 용량이 너무 커진다)
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

#전처리 특수기호 없애기
sampled_df['text']=sampled_df['text'].apply(clean_text)

sampled_df = sampled_df.reset_index(drop=True)

train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [4]:
#단어 10000개 선별
vect = CountVectorizer()
X_dtm = vect.fit_transform(to_txt_filter)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
import numpy as np
word_view=np.array(vect.get_feature_names_out())
sw=word_view[TorF]

In [5]:
#훈련데이터에서 선별한 10000개 단어 이외 단어는 <ukn>으로 변경
train_transe=[]
for i in range(len(x_train)):
    a=x_train[i].split()
    for j in range(len(a)):
        if a[j] not in sw:
            a[j] = '<ukn>'
    train_transe.append(' '.join(a))

In [6]:
#테스트데이터에서 선별한 10000개 단어 이외 단어는 <ukn>으로 변경
test_transe=[]
for i in range(len(x_test)):
    a=x_test[i].split()
    for j in range(len(a)):
        if a[j] not in sw:
            a[j] = '<ukn>'
    test_transe.append(' '.join(a))

In [7]:
# DTM 확인
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(train_transe)
# print(x_train.shape)

# 테스트데이터
x_test_dtm = dtmvector.transform(test_transe) #테스트 데이터를 DTM으로 변환

In [8]:
# TF-IDF Matrix확인
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

# 테스트데이터
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

(16000, 9935)


# 전통적인 머신러닝 모델로 학습

In [9]:
# SVM 모델
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'penalty': ['l1', 'l2']}

grid = GridSearchCV(LinearSVC(max_iter=500, dual=False), param_grid, refit=True, verbose=3)

grid.fit(tfidfv, y_train)
print(grid.best_params_)

predicted = grid.predict(tfidfv_test)
print("정확도:", accuracy_score(y_test, predicted))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .................C=0.1, penalty=l1;, score=0.842 total time=   0.1s
[CV 2/5] END .................C=0.1, penalty=l1;, score=0.843 total time=   0.1s
[CV 3/5] END .................C=0.1, penalty=l1;, score=0.837 total time=   0.1s
[CV 4/5] END .................C=0.1, penalty=l1;, score=0.844 total time=   0.1s
[CV 5/5] END .................C=0.1, penalty=l1;, score=0.857 total time=   0.1s
[CV 1/5] END .................C=0.1, penalty=l2;, score=0.897 total time=   0.0s
[CV 2/5] END .................C=0.1, penalty=l2;, score=0.892 total time=   0.0s
[CV 3/5] END .................C=0.1, penalty=l2;, score=0.892 total time=   0.0s
[CV 4/5] END .................C=0.1, penalty=l2;, score=0.899 total time=   0.0s
[CV 5/5] END .................C=0.1, penalty=l2;, score=0.903 total time=   0.0s
[CV 1/5] END ...................C=1, penalty=l1;, score=0.885 total time=   1.9s
[CV 2/5] END ...................C=1, penalty=l1;

C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5] END ..................C=10, penalty=l1;, score=0.855 total time=   2.1s


C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END ..................C=10, penalty=l1;, score=0.853 total time=   2.1s


...

[CV 3/5] END ................C=1000, penalty=l1;, score=0.852 total time=   2.5s


C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5] END ................C=1000, penalty=l1;, score=0.857 total time=   2.6s


C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END ................C=1000, penalty=l1;, score=0.853 total time=   2.6s
[CV 1/5] END ................C=1000, penalty=l2;, score=0.857 total time=   0.2s
[CV 2/5] END ................C=1000, penalty=l2;, score=0.859 total time=   0.2s
[CV 3/5] END ................C=1000, penalty=l2;, score=0.853 total time=   0.2s
[CV 4/5] END ................C=1000, penalty=l2;, score=0.862 total time=   0.1s
[CV 5/5] END ................C=1000, penalty=l2;, score=0.868 total time=   0.2s
{'C': 1, 'penalty': 'l2'}
정확도: 0.9125


In [10]:
#나이브 베이즈 모델
param_grid = {'alpha': [0.01, 0.1, 0.5, 1, 10, 100],
              'fit_prior': [True, False]}

grid = GridSearchCV(MultinomialNB(), param_grid, refit=True, verbose=3)

grid.fit(tfidfv, y_train)
print(grid.best_params_)

predicted = grid.predict(tfidfv_test)
print("정확도:", accuracy_score(y_test, predicted))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ........alpha=0.01, fit_prior=True;, score=0.913 total time=   0.0s
[CV 2/5] END ........alpha=0.01, fit_prior=True;, score=0.902 total time=   0.0s
[CV 3/5] END ........alpha=0.01, fit_prior=True;, score=0.905 total time=   0.0s
[CV 4/5] END ........alpha=0.01, fit_prior=True;, score=0.903 total time=   0.0s
[CV 5/5] END ........alpha=0.01, fit_prior=True;, score=0.912 total time=   0.0s
[CV 1/5] END .......alpha=0.01, fit_prior=False;, score=0.913 total time=   0.0s
[CV 2/5] END .......alpha=0.01, fit_prior=False;, score=0.902 total time=   0.0s
[CV 3/5] END .......alpha=0.01, fit_prior=False;, score=0.905 total time=   0.0s
[CV 4/5] END .......alpha=0.01, fit_prior=False;, score=0.903 total time=   0.0s
[CV 5/5] END .......alpha=0.01, fit_prior=False;, score=0.912 total time=   0.0s
[CV 1/5] END .........alpha=0.1, fit_prior=True;, score=0.911 total time=   0.0s
[CV 2/5] END .........alpha=0.1, fit_prior=True;

In [11]:
#logistic regression 모델
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'penalty': ['l1', 'l2']}

grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3)

grid.fit(tfidfv, y_train)
print(grid.best_params_)

predicted = grid.predict(tfidfv_test)
print("정확도:", accuracy_score(y_test, predicted))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END .................C=0.1, penalty=l2;, score=0.868 total time=   0.1s
[CV 2/5] END .................C=0.1, penalty=l2;, score=0.869 total time=   0.1s
[CV 3/5] END .................C=0.1, penalty=l2;, score=0.864 total time=   0.1s
[CV 4/5] END .................C=0.1, penalty=l2;, score=0.868 total time=   0.1s
[CV 5/5] END .................C=0.1, penalty=l2;, score=0.879 total time=   0.2s
[CV 1/5] END .....................C=1, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END .....................C=1, penalty=l

C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5] END ...................C=1, penalty=l2;, score=0.893 total time=   0.3s


C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5] END ...................C=1, penalty=l2;, score=0.892 total time=   0.3s


...

C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 4/5] END ................C=1000, penalty=l2;, score=0.876 total time=   0.4s


C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with 

[CV 5/5] END ................C=1000, penalty=l2;, score=0.880 total time=   0.3s
{'C': 1, 'penalty': 'l2'}
정확도: 0.913


C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
